In [ ]:
!pip install tensorflow==1.14.0 #run

In [ ]:
!unzip /content/models.zip

In [ ]:
#!rm -r /content/data

In [ ]:
!mkdir data

In [ ]:
#convert video to frames
#run
import cv2
vidcap = cv2.VideoCapture('/content/test 2.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*100)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("/content/data/frame"+str(count)+".jpg", image)
    return hasFrames
sec = 0
frameRate = 0.5
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#run
download = drive.CreateFile({'id': '1DpXhjXw9v4mwubR2bw8Yw72MhuqWDQQJ'})
download.GetContentFile('c3d_ucf101_finetune_whole_iter_20000_TF.model')

In [ ]:
#run
import shutil, sys
shutil.move("c3d_ucf101_finetune_whole_iter_20000_TF.model", "models/pretrained_models/") 

In [ ]:
#run
download = drive.CreateFile({'id': '1t8izbDP39J1UdUtKbv2UBeT9xXF28g5L'})
download.GetContentFile('checkpoint.zip')

In [ ]:
#run
!unzip checkpoint.zip

In [ ]:
#run
import cv2
import os
import numpy as np
#import matplotlib.pyplot as plt

class Video(object):

    def __init__(self, filename, isAnomaly, width=320, height=240):
        if filename is None:
            raise ValueError("invalid filename argument")
        self.isAnomaly=isAnomaly
        self.filename = filename
        self.frames=[]
        self.frameCount = 0
        self.width = width
        self.height = height
        self.__setFrames()


    def __split(self):
        print(self.filename)
        if os.path.exists(self.filename) is False:
            raise ValueError("Video File Not Found. Could not find: %s" % ('dataset/videos/'+self.filename))
        videoSource = cv2.VideoCapture(self.filename)
        success,image = videoSource.read() #success is true if file exist, and loads first frame in image
        count = 0
        dir = os.path.splitext(self.filename)[0] 
        if os.path.isdir(dir) is False:
            os.makedirs(dir)
        while success:
            resize_image = cv2.resize(image,(self.width, self.height))
            cv2.imwrite(dir+"/frame%d.jpg" %count,resize_image)
            success, image = videoSource.read()
            count+=1
        #print("Read %d Frames" %count)
        return count


    def __setFrames(self):

        """Sets frames from the video.
                Retrieves frames from the video and returns an image array.
                Image size is 240x320
                Before running, video should be in 30FPS.
                Args:
                Returns:

                Raises:
                    ValueError: Video File Not Found.
        """

        dir = os.path.splitext(self.filename)[0]  # gets the directory location
        if os.path.isdir(dir) is False:
            self.__split()

        count = len(os.listdir(dir)) #count list of files inside directory

        imageArr=[]
        for i in range(count):
            image = cv2.imread(dir+"/frame%d.jpg"%i)
            if image is not None: 
                imageArr.append(image)

        self.frames=imageArr
        self.frameCount = len(imageArr)

    def getFrames(self):
        return np.array(self.frames)

    def getAnomaly(self):
        return self.isAnomaly

    def getFilename(self):
        return self.filename

    def getFrameCount(self):
        return self.frameCount

    def resize(self, new_width, new_height):
        self.frames = [cv2.resize(frame, (new_width, new_height)) for frame in self.frames]

    def getSegments(self):
        count = self.getFrameCount()
        print("Getting segments:", count, " total frames.")
        frames = np.array(self.getFrames())
        print("FRAMES SHAPE: %s" % (str(frames.shape)))
        if count % 16 != 0:
            print("REMOVING FRAMES: %d" % (count%16))
            frames = frames[:-(count%16)]

        segments = frames.reshape([-1, 16, 112, 112, 3])
        
        print("SEGMENT SHAPE: %s" % (str(segments.shape)))
        return segments

    def __str__(self):
        return "You are print video object, use one of my methods, My filename is "+self.filename


In [ ]:
import requests
import datetime
def sms(pesttext):
    url = 'https://www.smsgateway.center/SMSApi/rest/send'
    f = '%Y-%m-%d %H:%M:%S.%f%z'
    from datetime import datetime, timedelta
    t = datetime.now() + timedelta(seconds=19830)
    out = t.strftime("%Y-%m-%d %H:%M:%S")
    print(out)
    querystring = {
        'userId': 'mitron',
        'password': 'Mitron@123',
        'senderId': 'MiHelp',
        'sendMethod': 'simpleMsg',
        'msgType': 'text',
        'mobile': '9686469235',
        'msg': pesttext,
        'duplicateCheck': 'true',
        'format': 'json',
        'scheduleTime': out,
        }
    headers = {'cache-control': 'no-cache'}
    response = requests.request('GET', url, headers=headers,
                                params=querystring)
    print(response.text)

In [ ]:
otp='252M2A'

In [ ]:
#from ucsd_dataset import ucsd_dataset
from models.c3d_model import C3DModel
from models.bc_model import BCModel
import numpy as np
import cv2
#from Video import Video

def main():
    # Load in one of the videos from the dataset
    folder = "data"#chaange location
    selected_video = Video(folder, 1)
    selected_video.resize(112,112)
    selected_video = selected_video.getSegments()

    c3d = C3DModel()
    features, elapsed_time = c3d.predict(selected_video)
    print("c3d",features)

    print(features.shape)
    print("Extracting features...")
    print("Elapsed time: %fs" % (elapsed_time))
    bc = BCModel()
    bc.load_model("checkpoint/model")
    scores, elapsed_time = bc.predict(features)
    #print("bc",scores)
    #print(scores.shape)

    url = 'data/'#change location
    img = []
    predictions = []
    dir = os.path.splitext("data/")[0]#change Location
    frames = len(os.listdir(dir))
    imageArr=[]
    for i in range(frames):
        image = cv2.imread(dir+"/frame%d.jpg"%i)
        if image is not None:
             imageArr.append(image)
    print("Images loaded!!!")        

    # Collect all the images into a Python list
    print("Gathering images...")
   # for k in range(0, frames+1):
   #     print("Progress: %2.1f" % (float(k) / frames))
   #     img.append(cv2.imread(url + '/frame' + str(k) + '.jpg'))

    height, width, layers = imageArr[0].shape

    print("Predicting scores...")
    print("Elapsed time: %fs" % (elapsed_time))

    # Iterate through and print out each 16-frame anomaly score and
    cou = 0
    for i in range(len(scores)):
        w = i*16
        # Ask use for the name of the video and how many frames to use

        if scores[i] > 0.5:
          for w in range((i*16),(i+1)*16):
              font = cv2.FONT_HERSHEY_SIMPLEX 
              #cv2.rectangle(img[w],(384,0),(240,240),(0,255,0),3)
              print("Printing Text...")
              
              cv2.putText(imageArr[w],'Risk Detected',(100,100), font, 0.8, (0,0,255), 2, cv2.LINE_AA)
              if cou == 0:
                 #sms("Risk Detected ,Kindly click the location https://goo.gl/maps/Vmk6UpgfW2aSPVrX6")
                 #sms("Crime/ Anomaly Detected ,Kindly click the location  https://goo.gl/maps/iXA4gjp4fsdKvqat6")
                 sms('OTP is:'+str(otp)+'--MitronTech')
                 cou = cou +1

        print("Frames (%4d to %4d)\tScore: %f\t%s" % (i*16,(i+1)*16,scores[i], "Risk detected!" if scores[i] > .5 else ""))
        predictions.append(scores[i])
        

    # Convert the list of images to a video and saves it to the working directory
    print("Creating video writer...")
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter('Risk.avi', fourcc, 30.0, (width, height))
    print("Video writer created!")

    print("Total Frames: %d" % (len(img)))

    # Writes each image to the video
    print("Writing Frames...")
    try:
        for j in range(0, frames):
            print("Progress: %2.1f" % (float(j)/frames))
            video.write(imageArr[j])
    except (ValueError,IndexError):
        print("index out of range")

    # Finalization
    print("Finalizing...")
    cv2.destroyAllWindows()
    video.release()
    print("Finished!!!")
    results = ""
    predd = predictions
    if max(predd) >= 0.5:
        print("Risk is detected in this Video")
        #flash("Risk is detected in uploaded Video. Video is sent to Mail")
    else:
        print("Risk is not detected in this Video.")

In [ ]:
if __name__ == '__main__':
  main()

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = [0,1,1,1,0,1,1,1,1,0,0,1,1,0]
preds = [1,0,1,1,0,1,1,1,1,0,1,1,1,0]
fpr, tpr, threshold = metrics.roc_curve(probs, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()